In [3]:
!pip install d2l
import torch
from torch import nn
from d2l import torch as d2l

     |████████████████████████████████| 92kB 4.6MB/s 


In [4]:
#11.10.1. 基本操作
#我们可以对输入矩阵X和卷积核矩阵 K实现基本的转置卷积运算trans_conv。
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i:i + h, j:j + w] += X[i, j] * K
    return Y

In [5]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [6]:
#或者，当输入X和卷积核K都是四维张量时，我们可以使用高级API获得相同的结果。
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<SlowConvTranspose2DBackward>)

In [7]:
#11.10.2. 填充、步幅和多通道
#与常规卷积不同，在转置卷积中，填充被应用于的输出（常规卷积将填充应用于输入）。 
#例如，当将高和宽两侧的填充数指定为1时，转置卷积的输出中将删除第一和最后的行与列。
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<SlowConvTranspose2DBackward>)

In [8]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<SlowConvTranspose2DBackward>)

In [9]:
#同样，如果我们将  X  代入卷积层  f  来输出  Y=f(X)  ，
#并创建一个与  f  具有相同的超参数、但输出通道数量是  X  中通道数的转置卷积层  g ，
#那么  g(Y)  的形状将与  X  相同。 下面的示例可以解释这一点。
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

In [10]:
#11.10.3. 与矩阵变换的联系
#转置卷积为何以矩阵变换命名呢？ 让我们首先看看如何使用矩阵乘法来实现卷积。 
#在下面的示例中，我们定义了一个 3×3 的输入X和 2×2 卷积核K，然后使用corr2d函数计算卷积输出Y。
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Y

tensor([[27., 37.],
        [57., 67.]])

In [11]:
#接下来，我们将卷积核K重写为包含大量0的稀疏权重矩阵W。 权重矩阵的形状是（ 4 ， 9 ），其中非0元素来自卷积核K。
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [12]:
#逐行连接输入X，获得了一个长度为9的矢量。 然后，W的矩阵乘法和向量化的X给出了一个长度为4的向量。 
#重塑它之后，可以获得与上面的原始卷积操作所得相同的结果Y：我们刚刚使用矩阵乘法实现了卷积。
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

In [13]:
#同样，我们可以使用矩阵乘法来实现转置卷积。 在下面的示例中，我们将上面的常规卷积 2×2 的输出Y作为转置卷积的输入。 
#想要通过矩阵相乘来实现它，我们只需要将权重矩阵W的形状转置为 (9,4) 。
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])